In [21]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, './fma_metadata/')
from fma_metadata import utils
import warnings
warnings.filterwarnings('ignore')

In [22]:
# extract labels for genres
tracks = utils.load('/Users/gaetanoantonicchio/Desktop/dataset_DM2/fma_metadata/tracks.csv')
genres_labels = tracks['track'][['genre_top']]
genres_labels.dropna(inplace=True)
#genres_labels.head()
# exporting labels: song_popularity
with open("genre_labels.csv","w") as new_label:
    new_label.write(genres_labels.to_csv(index=True))

In [23]:
# extract labels for song popularity
echonest = utils.load('/Users/gaetanoantonicchio/Desktop/dataset_DM2/fma_metadata/echonest.csv')
social_features_dataset= echonest['echonest','social_features']
social_features_dataset['song_popularity'] = np.where( ( (social_features_dataset['song_hotttnesss'] >= 0.15)\
                                                         & (social_features_dataset['artist_hotttnesss'] >= 0.4 ) ),
                                                       'popular', 'not popular')

song_popularity_labels = social_features_dataset[['song_popularity']]
#song_popularity_labels.head()
# exporting labels: song_popularity
with open("song_popularity_labels.csv","w") as new_label:
    new_label.write(song_popularity_labels.to_csv(index=True))

In [24]:
# extract dataset_song_popularity.csv
dataset_song_popularity = echonest['echonest','audio_features'].merge(tracks['track'][['listens','bit_rate']], on='track_id')

with open("dataset_song_popularity.csv","w") as new_label:
    new_label.write(dataset_song_popularity.to_csv(index=True))

In [25]:
# extract labels for years_of_rock

rock_genre = tracks['track'][['genre_top']][tracks['track']['genre_top'] == 'Hip-Hop']
# filtered echonest_audio_features by rock genres
rock_df = echonest['echonest','audio_features'].merge(rock_genre,on='track_id')

In [26]:
rock_df.shape

(910, 9)

In [31]:
tracks['artist'].columns

Index(['active_year_begin', 'active_year_end', 'associated_labels', 'bio',
       'comments', 'date_created', 'favorites', 'id', 'latitude', 'location',
       'longitude', 'members', 'name', 'related_projects', 'tags', 'website',
       'wikipedia_page'],
      dtype='object')

In [40]:
# extract year from date_created --> to new col: year_created in track
rock_df = echonest['echonest','audio_features'].merge(tracks['track'][['genre_top','listens','duration','bit_rate','comments']],on='track_id')
rock_df = rock_df[rock_df['genre_top'] == 'Rock']

tracks['track','year'] = pd.DatetimeIndex(tracks['track','date_created']).year
years = tracks['track'][['year']]
rock_df = rock_df.merge(years, on='track_id')
rock_df = rock_df.merge(echonest['echonest','social_features'][['song_hotttnesss','artist_hotttnesss','artist_familiarity']], on ='track_id')
rock_df = rock_df.merge(tracks['artist'][['name']],on='track_id')

In [41]:
rock_df.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,genre_top,listens,duration,bit_rate,comments,year,song_hotttnesss,artist_hotttnesss,artist_familiarity,name
track_id,,,,,,,,,,,,,,,,,,
153,0.988306,0.255661,0.979774,0.973006,0.121342,0.051740,90.241,0.034018,Rock,424,405,256000,0,2008,0.044300,0.274455,0.173315,Arc and Sender
154,0.970135,0.352946,0.023852,0.957113,0.113261,0.032177,53.758,0.035632,Rock,205,319,256000,0,2008,0.070213,0.274455,0.173315,Arc and Sender
155,0.981657,0.142249,0.912122,0.967294,0.363510,0.087527,91.912,0.034325,Rock,197,756,192000,0,2008,0.000000,0.274455,0.173315,Arc and Sender
169,0.989141,0.225978,0.722835,0.263076,0.092371,0.053406,94.322,0.028347,Rock,270,144,192000,0,2008,0.039431,0.287989,0.162622,Argumentix
170,0.886660,0.298518,0.744333,0.920950,0.139587,0.088781,97.880,0.073548,Rock,122,181,192000,0,2008,0.000000,0.287989,0.162622,Argumentix


In [42]:
with open("rock_year_df.csv","w") as new_label:
    new_label.write(rock_df.to_csv(index=True))